In [1]:
from pathlib import Path
from typing import Tuple, Union, List, Dict, Iterable, Optional

import torch
from tqdm.notebook import tqdm

from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
# from evaluations.helpers import GroundedWord, load_recomposers_en_masse
# from evaluations.clustering import graph_en_masse
# from evaluations.euphemism import cherry_words

In [8]:
from dataclasses import dataclass

BASE_DIR = Path.home() / 'Research/congressional_adversary/results'
sup_PE = torch.load(BASE_DIR / 'SGNS deno/pretrained super large/init.pt')['model']
WTI = sup_PE.word_to_id
ITW = sup_PE.id_to_word
sup_PE = sup_PE.embedding.weight.detach().cpu().numpy()
print(f'Vocab size = {len(WTI):,}')


sub_PE = torch.load(BASE_DIR / 'bill topic/pretrained subset/init.pt')['model']
sub_PE_WID = sub_PE.word_to_id
sub_PE_GD = sub_PE.grounding
del sub_PE


@dataclass
class GroundedWord():
    word: str

#     def __post_init__(self) -> None:
#         self.word_id: int = WTI[self.word]
#         metadata = sub_PE_GD[self.word]
#         self.freq: int = metadata['freq']
#         self.R_ratio: float = metadata['R_ratio']
#         self.majority_deno: int = metadata['majority_deno']

#         self.PE_neighbors = self.neighbors(sup_PE)
            
#     def neighbors(self, embed, top_k=10): 
#         query_id = sup_PE.word_to_id[self.word]
#         query_vec = sup_PE[query_id]
#         distances = [
#             distance.cosine(query_vec, neighbor_vec)
#             for neighbor_vec in sup_PE]
#         self.sup_PE_neighbors = set()
#         for sort_rank, neighbor_id in enumerate(sorted_neighbor_indices):
#             if num_neighbors == top_k:
#                 break
#             if query_id == neighbor_id:
#                 continue
#             neighbor_word = self.id_to_word[neighbor_id]
#             if editdistance.eval(query_word, neighbor_word) < 3:
#                 continue
            
#         self.sub_PE_neighbors: List[str] = nearest(, sub_PE)

    def __str__(self) -> str:
        return str(vars(self))
    
    
capitalism: List[GroundedWord] = []
socialism: List[GroundedWord] = []
for word in sub_PE_WID.keys():
    ratio = sub_PE_GD[word]['R_ratio']
    freq = sub_PE_GD[word]['freq']
    word = GroundedWord(word)
    if ratio < 0.2 and freq > 100:  # 0.2:
        socialism.append(word)
    elif ratio > 0.8 and freq > 100:  # 0.8:
        capitalism.append(word)

print(
    f'{len(capitalism)} capitalists\n'
    f'{len(socialism)} socialists')
polarization = capitalism + socialism

Vocab size = 111,386
29 capitalists
64 socialists


In [3]:
import numpy as np

def get_embed(model: Decomposer) -> np.ndarray:
    return model.embedding.weight.detach().cpu().numpy()


def load(
        path: Path,
        match_vocab: bool = False,
        device: str = 'cpu'
        ) -> np.ndarray:
    model = torch.load(path, map_location=device)['model']
    try:
        assert model.word_to_id == WTI
    except AssertionError:
        print(f'Vocabulary mismatch: {path}')
        print(f'Vocab size = {len(model.word_to_id)}')
        if match_vocab:
            raise RuntimeError
        else:
            return None
    return get_embed(model)


def load_decomposers_en_masse(
        in_dirs: Union[Path, List[Path]],
        patterns: Union[str, List[str]]
        ) -> Tuple[Dict[str, np.ndarray], ...]:
    if not isinstance(in_dirs, List):
        in_dirs = [in_dirs, ]
    if not isinstance(patterns, List):
        patterns = [patterns, ]
    checkpoints: List[Path] = []
    for in_dir in in_dirs:
        for pattern in patterns:
            checkpoints += list(in_dir.glob(pattern))
    if len(checkpoints) == 0:
        raise FileNotFoundError('No model with path pattern found at in_dir?')

    models = {
#         'pretrained superset': load(BASE_DIR / 'bill topic/pretrained superset/init.pt'),
#         'pretrained subset': load(BASE_DIR / 'bill topic/pretrained subset/init.pt')
    }
    for path in tqdm(checkpoints):
        tqdm.write(f'Loading {path}')
        embed = load(path) 
        if embed is None:
            continue
#         name = path.parent.name + path.name 
        name = path.parent.name 
        models[name] = embed
    return models

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

sns.set()

def plot(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path
        ) -> None:
    fig, ax = plt.subplots(figsize=(15, 10))
#     skew = [w.R_ratio for w in words]
#     freq = [w.freq for w in words]
    sns.scatterplot(
        coordinates[:, 0], coordinates[:, 1],
#         hue=skew, palette='coolwarm',  # hue_norm=(0, 1),
#         size=freq, sizes=(200, 1000),
        legend=None, ax=ax)
    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=20)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def plot_categorical(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path,
        fancy: bool = True
        ) -> None:
    if fancy:
        fig, ax = plt.subplots(figsize=(20, 10))
        categories = [w.majority_deno for w in words]
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1],
            hue=categories, palette='muted', hue_norm=(0, 1),
            size=freq, sizes=(200, 1000),
            legend='brief', 
            ax=ax)
        chartBox = ax.get_position()
        ax.set_position(  # adjust legend
            [chartBox.x0, chartBox.y0, chartBox.width * 0.6, chartBox.height])
        ax.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
    else:
        fig, ax = plt.subplots(figsize=(20, 10))
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1], ax=ax)

    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=12)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def graph_en_masse(
        models: Dict[str, np.ndarray],
        out_dir: Path,
        reduction: str,  # 'PCA', 'TSNE', or 'both'
        words: List[GroundedWord],
        # hues: Union[List[float], List[int]],
        # sizes: List[int],
        perplexity: Optional[int] = None,
        categorical: bool = False
        ) -> None:
    Path.mkdir(out_dir, parents=True, exist_ok=True)
    word_ids = np.array([w.word_id for w in words])
    for model_name, embed in tqdm(models.items()):
        space = embed[word_ids]
        if reduction == 'PCA':
            visual = PCA(n_components=2).fit_transform(space)
        elif reduction == 'TSNE':
            assert perplexity is not None
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        elif reduction == 'both':
            assert perplexity is not None
            space = PCA(n_components=30).fit_transform(space)
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        else:
            raise ValueError('unknown dimension reduction method')
        if categorical:
            plot_categorical(visual, words, out_dir / f'{model_name}.png')
        else:
            plot(visual, words, out_dir / f'{model_name}.png')


In [ ]:
cherry_words = [
    'government', 'washington',
    'estate_tax', 'death_tax',
    'public_option', 'governmentrun',
    'foreign_trade', 'international_trade',
    'cut_taxes', 'trickledown'
]

cherry_words = [GroundedWord(w) for w in cherry_words]

In [9]:
base_dir = Path('../../results/SGNS deno/sans recomposer')
# deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch10.pt')
deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch*.pt')

Loading ../../results/SGNS deno/sans recomposer/L4 BS4/epoch10.pt
Loading ../../results/SGNS deno/sans recomposer/L4/epoch10.pt



In [ ]:
models = deno_space
stuff = polarization

graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p5',
    reduction='TSNE', perplexity=5, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p3',
    reduction='TSNE', perplexity=3, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p2',
    reduction='TSNE', perplexity=2, words=stuff, categorical=True)

In [6]:
from scipy.spatial.distance import cosine as cos_dist
import editdistance

def vec(query: str, embed: np.ndarray) -> np.ndarray:
    try:
        query_id = WTI[query]
    except KeyError:
        raise KeyError(f'Out of vocabulary: {query}')
    return embed[query_id]


def nearest_neighbors(
        query: str,
        embed: np.ndarray,
        top_k: int = 10
        ) -> None:
    query_vec = vec(query, embed)
    print(f"{query}'s neareset neighbors:")
    distances = [
        cos_dist(query_vec, neighbor_vec)
        for neighbor_vec in embed]
    neighbor_indices = np.argsort(distances)
    num_neighbors = 0        
    for sort_rank, neighbor_id in enumerate(neighbor_indices):
        if num_neighbors == top_k:
            break
#         if query_id == neighbor_id:
#             continue
        neighbor_word = ITW[neighbor_id]

        if editdistance.eval(query, neighbor_word) < 3:
            continue
        cosine_similarity = 1 - distances[neighbor_id]
        # neighbor_ids.append(neighbor_id)
        num_neighbors += 1
        print(f'{cosine_similarity:.4f}\t{neighbor_word}')
    print()

In [ ]:
deno_space['bill topic'] = 

In [10]:
deno_space.keys()

dict_keys(['L4 BS4', 'L4'])

In [11]:
query = 'death_tax'
nn = nearest_neighbors
our_model = deno_space['L4']
nn(query, sup_PE)
nn(query, our_model)

death_tax's neareset neighbors:
0.8923	estate_tax
0.8097	estate_taxes
0.7626	repeal_the_death
0.7265	inheritance
0.7078	estates
0.7058	marriage_penalty
0.7054	estate_tax_relief
0.7026	confiscatory
0.6846	taxable_event
0.6762	marriage_penalty_tax

death_tax's neareset neighbors:
0.5959	familyowned
0.5730	inheritance
0.5555	estate_tax
0.5261	estate_taxes
0.5225	family_farms
0.5057	family_farm
0.5025	estate
0.4966	heirs
0.4695	marriage_penalty
0.4639	undertaker



In [12]:
query = 'estate_tax'
nn(query, sup_PE)
nn(query, our_model)

estate_tax's neareset neighbors:
0.8923	death_tax
0.8697	estates
0.8360	death_taxes
0.8125	estates_would
0.7943	estate_tax_relief
0.7392	inheritance
0.7075	repeal_the_death
0.7045	estate
0.6890	family_farms
0.6844	marriage_penalty

estate_tax's neareset neighbors:
0.8851	estates
0.7540	estate_tax_relief
0.6820	family_farms
0.6472	familyowned
0.6428	democratic_alternative
0.6308	estate
0.6284	inheritance
0.6275	estates_would
0.6066	wealthiest
0.5702	death_taxes



In [13]:
query = 'public_option'
nn(query, sup_PE)
nn(query, deno_space['L4'])

public_option's neareset neighbors:
0.8769	public_plan
0.8208	government_option
0.8003	governmentrun_plan
0.7969	government_plan
0.7803	insurance_plan
0.7760	governmentrun_health
0.7517	public_health_insurance
0.7455	private_health_insurance
0.7432	lower_costs
0.7348	private_market

public_option's neareset neighbors:
0.7612	public_plan
0.6937	insurance_plan
0.6787	private_health_insurance
0.6599	private_insurance
0.6548	lower_costs
0.6499	health_insurance_companies
0.6398	insurance_market
0.6372	singlepayer
0.6334	health_insurance_industry
0.6280	health_reform



In [14]:
query = 'foreign_trade'
nn(query, sup_PE)
nn(query, deno_space['L4'])

foreign_trade's neareset neighbors:
0.7665	unfair_trade_practices
0.7424	international_trade
0.7331	unfair_trade
0.7287	us_trade
0.7261	trade_practices
0.7219	unfair_trading_practices
0.7146	trade
0.7077	us_exports
0.6895	foreign_competition
0.6880	trading_partners

foreign_trade's neareset neighbors:
0.7244	unfair_trade_practices
0.6509	trade_laws
0.6469	unfair_trade
0.6448	trade_practices
0.6325	trading_partners
0.6280	trade_problems
0.6245	us_exports
0.6224	us_trade
0.6209	foreign_competition
0.6158	foreign_markets



In [15]:
query = 'political_speech'
nn(query, sup_PE)
nn(query, deno_space['L4'])

political_speech's neareset neighbors:
0.8270	free_speech
0.7660	the_first_amendment
0.7543	first_amendment_rights
0.7400	first_amendment
0.7334	freedom_of_speech
0.7286	issue_advocacy
0.7039	buckley_versus_valeo
0.6988	buckley_v_valeo
0.6820	campaign_spending
0.6659	the_first_amendments

political_speech's neareset neighbors:
0.6674	buckley_v_valeo
0.6672	free_speech
0.6569	issue_advocacy
0.6482	the_first_amendment
0.6435	first_amendment_rights
0.6271	buckley_versus_valeo
0.6209	campaign_spending
0.6170	mccainfeingold
0.6061	first_amendment
0.5947	buckley



In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p5',
    reduction='TSNE', perplexity=5, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p3',
    reduction='TSNE', perplexity=3, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p2',
    reduction='TSNE', perplexity=2, words=cherry_words, categorical=False)

In [ ]:
models = deno_space

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p5',
    reduction='TSNE', perplexity=5,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p3',
    reduction='TSNE', perplexity=3,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p10',
    reduction='TSNE', perplexity=10,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

In [ ]:
models = cono_space

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/topic/PCA',
#     reduction='PCA',
#     word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

# Homogeneity V-Measure

In [ ]:
# Deno space, eval deno, higher is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=10)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Deno space, eval cono, lower is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Cono space, eval cono, higher is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')

In [ ]:
# Cono space, eval deno, lower is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')